In [24]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import sklearn.metrics as sk_metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import tempfile
import statsmodels.api as sm
import os
from tensorflow import keras
import statsmodels.formula.api as smf

from tensorflow.keras import layers
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
import pymssql
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from tqdm import tqdm
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from mpl_toolkits import mplot3d


random_state = 2023

# Preset matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

print(tf.__version__)
# To make the results reproducible, set the random seed value.
tf.random.set_seed(22)

2.9.0


In [25]:
def get_column_index(cursor, column_name):
    for index, col in enumerate(cursor.description):
        if col[0] == column_name:
            return index
additional_data_headers = ["RentMortgagePayment" ,"TotalIncome" ,"RentMortgage" ,"CarPayment" ,"Expenses" ,"AvailableIncome" ,"Miles"]
additional_data_sql = "VIN,"
for i, header in enumerate(additional_data_headers):
    additional_data_sql += header
    if i!=len(additional_data_headers)-1:
        additional_data_sql+=','

In [49]:
conn = pymssql.connect(server='localhost', user='sa', password='Passw0rd!', database='Loans_2023_01_27')
cursor = conn.cursor()

headers = []
rows = []

# Underwriting Data
cursor.execute("SELECT * FROM dbo.UnderwritingData")

for entity in cursor.description:
    headers.append(entity[0])

vin_index = get_column_index(cursor, "VIN")

res = cursor.fetchone()
while res:
    row = [element for element in res]
    if row[vin_index] is not None:
        rows.append(row)
    res = cursor.fetchone()

df = pd.DataFrame(rows, columns=headers)
ids = [i for i in range(0, len(df))]
default_status = ["NA" for _ in range(len(df))]
default_delq = ["NA" for i in range(len(df))]
default_apr = ["NA" for _ in range(len(df))]
df["ID"] = ids
df["APR"] = default_apr
df["Status"] = default_status
df["Delq"] = default_delq
df = df[df['OpeningBalance']!=0]


#Raw algorithm data
headers = []
rawAlgRows =[]
cursor.execute("Select * FROM dbo.RawAlgorithmData")

for entity in cursor.description:
    headers.append(entity[0])

vin_index = get_column_index(cursor, "Vin6")

res = cursor.fetchone()
while res:
    row = [element for element in res]
    if row[vin_index] is not None:
        rawAlgRows.append(row)
    res = cursor.fetchone()

rawAlgDf = pd.DataFrame(rawAlgRows, columns=headers)
#prune extra space from end of vins
rawAlgDf['Vin6'] = rawAlgDf['Vin6'].apply(lambda x: x.strip())
df['VIN'] = df['VIN'].apply(lambda x: x.strip())


#matching by credit score
for idx, row in tqdm(rawAlgDf.iterrows()):
    vin = row["Vin6"]
    vinsFromUnder = df[df["VIN"]==vin]


    for index, match in vinsFromUnder.iterrows():
        matchID = match["ID"]
        underwritingCreditScore = (int)(match["CreaditScore"])
        rawCreditScore = (int)(row["Score"])

        #we don't want to match if the scores are zero, since that means they are missing
        if rawCreditScore==0:
            continue

        if underwritingCreditScore==rawCreditScore:
            df.loc[df["ID"]==matchID,"Status"]=row["ProcStatsCd"]
            df.loc[df["ID"]==matchID,"Delq"]=row["Delq"]
            df.loc[df["ID"]==matchID, "APR"] = int(row["Apr"] * 100) if row["Apr"] is not None else None #convert apr to an int

df = df[(df['Status']=='CO') | (df['Status']=='CL')]
df['Status'].replace(to_replace=['CO', 'CL'], value=[0,1], inplace=True)
df['HomeOwner'].replace(to_replace=['N', 'Y'], value=[0,1], inplace=True)
df['DelqPct'] = df.apply(lambda row : (100*row['Delq'])/row['OpeningBalance'],axis=1)
df.fillna(df.mean(), inplace=True)

#TODO maybe put back: CreaditScore, CurrentPaymentsMade, DelqPct
df = df[["OpeningBalance","Salary","Term","VehicleYear","LTV","DTI","TradelinesTotal","TradelinesBalance","Reposessions","Bankrupcies","Chageoff","Mortgage","NinetyDaysDelinquent","InquiresTotal","OpenTotal","ClosedTotalTermsNotPaid","LatePaymentHistory001230","LatePaymentHistory001260","LatePaymentHistory001290Plus","PaidAccounts","InquiriesLast6Months","OpenTotalInBadStatus","RentMortgage","CashDown","HomeOwner", "APR", "Status"]]

#outlier handling
for col in df.columns:
    if(col in ["Status", "DelqPct", "CreaditScore", "HomeOwner", "Reposessions", "Chageoff", "APR"]):
        continue
    percentiles = df[col].quantile([0.00, 0.999]).values
    df.loc[df[col]<=percentiles[0], col] = percentiles[0]
    df.loc[df[col]>=percentiles[1], col] = percentiles[1]

26508it [00:30, 859.65it/s] 
/var/folders/kh/np0tdw052gg77h97rt6rm3l80000gn/T/ipykernel_70814/4149204941.py:81: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [50]:
#standarizing
# from sklearn.preprocessing import MinMaxScaler
# X = MinMaxScaler().fit_transform(X)
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder()
# y = ohe.fit_transform(y).toarray()

X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1], df.iloc[:,-1:], test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# sc_y = StandardScaler()
# y_train = y_train.values.reshape((len(y_train), 1))
# y_train = sc_y.fit_transform(y_train)
# y_train = y_train.ravel()

print(X_train[:10])
# print(y_train[:10])

[[-0.5045333  -0.46238504  0.43762614  0.0706154   0.36812794 -0.3398845
  -0.51068203 -0.48779484 -0.38535294 -0.07440166 -0.87687381 -0.18950581
  -0.14813506 -1.02116273 -0.30462629 -0.21213377 -0.65848734 -0.64790462
  -0.47205863 -0.22641098 -0.96734764 -0.14792522  0.7143976  -0.56744881
  -0.18950581  0.12963191]
 [-1.29619547 -0.59186782 -1.60872263  0.0706154   0.24155284 -0.36223408
  -0.18149564 -0.3312031  -0.38535294 -0.07440166 -0.37555798 -0.18950581
   0.74319505 -0.71988724  0.23387006  0.2659207  -0.65848734 -0.64790462
   0.24573388 -0.4427535  -0.68610366  0.74333914  0.62543474 -0.56744881
  -0.18950581 -0.6233156 ]
 [ 1.44337812  0.55239855  0.43762614  0.12803206 -1.37006257  0.22891219
   0.03796195 -0.11681309  2.1414345  -0.07440166 -0.37555798 -0.18950581
   0.07469747 -0.79520611 -0.48412507 -0.55075569  0.30149169  0.73669537
  -0.47205863  0.63895906 -0.82672565  0.07489087 -0.5268462  -0.95585575
  -0.18950581 -1.23936356]
 [ 0.4378291  -0.35096963  0.437

In [59]:
# -*- coding: utf-8 -*-
"""
MIT Licence

Zoghbi Abderraouf
Change data to your location
"""

from keras import backend as K
# from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer, InputSpec
from keras.initializers import RandomUniform, Initializer, Constant
import numpy as np


class InitCentersRandom(Initializer):
    """ Initializer for initialization of centers of RBF network
        as random samples from the given data set.
    # Arguments
        X: matrix, dataset to choose the centers from (random rows
          are taken as centers)
    """

    def __init__(self, X):
        self.X = X

    def __call__(self, shape, dtype=None):
        assert shape[1] == self.X.shape[1]
        idx = np.random.randint(self.X.shape[0], size=shape[0])
        return self.X[idx, :]


class RBFLayer(Layer):
    """ Layer of Gaussian RBF units.
    # Example
    ```python
        model = Sequential()
        model.add(RBFLayer(10,
                           initializer=InitCentersRandom(X),
                           betas=1.0,
                           input_shape=(1,)))
        model.add(Dense(1))
    ```
    # Arguments
        output_dim: number of hidden units (i.e. number of outputs of the
                    layer)
        initializer: instance of initiliazer to initialize centers
        betas: float, initial value for betas
    """

    def __init__(self, output_dim, initializer=None, betas=1.0, **kwargs):
        self.output_dim = output_dim
        self.init_betas = betas
        if not initializer:
            self.initializer = RandomUniform(0.0, 1.0)
        else:
            self.initializer = initializer
        super(RBFLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self.centers = self.add_weight(name='centers',
                                       shape=(self.output_dim, input_shape[1]),
                                       initializer=self.initializer,
                                       trainable=True)
        self.betas = self.add_weight(name='betas',
                                     shape=(self.output_dim,),
                                     initializer=Constant(
                                         value=self.init_betas),
                                     # initializer='ones',
                                     trainable=True)

        super(RBFLayer, self).build(input_shape)

    def call(self, x):

        C = K.expand_dims(self.centers)
        H = K.transpose(C-K.transpose(x))
        return K.exp(-self.betas * K.sum(H**2, axis=1))

        # C = self.centers[np.newaxis, :, :]
        # X = x[:, np.newaxis, :]

        # diffnorm = K.sum((C-X)**2, axis=-1)
        # ret = K.exp( - self.betas * diffnorm)
        # return ret

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        # have to define get_config to be able to use model_from_json
        config = {
            'output_dim': self.output_dim
        }
        base_config = super(RBFLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

from keras.initializers import Initializer
from sklearn.cluster import KMeans


class InitCentersKMeans(Initializer):
    """ Initializer for initialization of centers of RBF network
        by clustering the given data set.
    # Arguments
        X: matrix, dataset
    """

    def __init__(self, X, max_iter=100):
        self.X = X
        self.max_iter = max_iter

    def __call__(self, shape, dtype=None):
        assert shape[1] == self.X.shape[1]

        n_centers = shape[0]
        km = KMeans(n_clusters=n_centers, max_iter=self.max_iter, verbose=0)
        km.fit(self.X)
        return km.cluster_centers_

# Commented out IPython magic to ensure Python compatibility.
import numpy as np, pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Activation
from keras.optimizers import RMSprop
from keras.optimizers import Adam
import matplotlib.pyplot as plt



# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.transform(X_test)

# sc_y = StandardScaler()
# y_train = y_train.reshape((len(y_train), 1))
# y_train = sc_y.fit_transform(y_train)
# y_train = y_train.ravel()

# print(X_train.shape)

#initializer=InitCentersKMeans(X_train),
model = Sequential()
rbflayer = RBFLayer(50,
                        betas=1.0,
                        input_shape=(26,))
model.add(rbflayer)
model.add(Dense(1))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.00000001)
model.compile(loss='mean_squared_error',
                  optimizer=opt, metrics=['accuracy'])
print(model.summary())
history1 = model.fit(X_train, y_train, epochs=3, batch_size=32)

# import matplotlib.pyplot as plt
# plt.plot(history1.history['accuracy'])
# plt.plot(history1.history['loss'])
# plt.title('train accuracy and loss')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['accuracy', 'loss'], loc='upper left')
# plt.show()

# # saving to and loading from file
# z_model = f"Z_model.h5"
# print(f"Save model to file {z_model} ... ", end="")
# model.save(z_model)
# print("OK")

# #model already saved in file
# from tensorflow.keras.models import  load_model
# newmodel1= load_model("Zoghbio.h5",
#                           custom_objects={'RBFLayer': RBFLayer})
# print("OK")
#
# # Evaluate the model on the test data using `evaluate`
# print("Evaluate on test data")
# results = newmodel1.evaluate(X_test, y_test, batch_size=32)
# print("test loss:", results[0])
# print("test accuracy:",results[1]*100,'%')

y_pred = model.predict(X_test)
print(y_pred)
#Converting predictions to label
# pred = list()
# for i in range(len(y_pred)):
#     pred.append(np.argmax(y_pred[i]))
# #Converting one hot encoded test label to label
# test = list()
# for i in range(len(y_test)):
#     test.append(np.argmax(y_test[i]))

# from sklearn.metrics import accuracy_score
# a = accuracy_score(pred,test)
# print('Test Accuracy is:', a*100)

(6721, 26)
Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rbf_layer_30 (RBFLayer)     (None, 50)                1350      
                                                                 
 dense_28 (Dense)            (None, 1)                 51        
                                                                 
 activation_28 (Activation)  (None, 1)                 0         
                                                                 
Total params: 1,401
Trainable params: 1,401
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3


2023-04-04 15:31:17.028597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


211/211 [==============================] - 3s 11ms/step - loss: 0.4900 - accuracy: 0.5100
Epoch 2/3
211/211 [==============================] - 2s 8ms/step - loss: 0.4900 - accuracy: 0.5100
Epoch 3/3
29/53 [===============>..............] - ETA: 0s

2023-04-04 15:31:23.350762: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - 0s 4ms/step
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
